In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset

In [ ]:
dataset = load_dataset("dkagramanyan/horoscopes_ru")

In [14]:
def dataset2gpt(dataset, file_name):
    with open(f'{file_name}.txt', 'w', encoding='utf-8') as f:
        for line in dataset:
            date = line['date']
            date = date.split('-')
            date = date[2] + '-' + date[1] + '-' + date[0]
            sign = line['sign'].capitalize()
            text = line['text']
            context = '<s>' + sign + ' ' + date + '\n' + text + '</s>\n'
            f.write(context)
        f.close()

In [15]:
dataset2gpt(dataset['train'],'train_horoscopes')
dataset2gpt(dataset['test'],'test_horoscopes')

[Дообучение](https://github.com/sberbank-ai/ru-gpts/blob/master/examples/Finetune_RuGPTs_with_HF.ipynb) GPT3small на своем наборе данных

python ru-gpts/pretrain_transformers.py --output_dir=models/horoscopes --model_type=gpt2 --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 --do_train --train_data_file=train_horoscopes.txt --do_eval --eval_data_file=test_horoscopes.txt --per_gpu_train_batch_size 15 --gradient_accumulation_steps 1 --num_train_epochs 10 --block_size 2048 --overwrite_output_dir

python ru-gpts/generate_transformers.py --model_type=gpt2 --model_name_or_path=models/essays --k=5 --p=0.95 --length=50 --repetition_penalty=5

# Prepare training dataset

In [ ]:
signs_r = ['овен', 'близнецы', 'телец', 'рак', 'лев', 'дева', 'весы', 'скорпион', 'стрелец', 'козерог', 'водолей',
           'рыбы']

In [ ]:
text = data.values[:, 2]
vals = np.zeros((text.shape[0]))
for i, line in enumerate(text):
    vals[i] = len(line.split())
print(vals.max())
print(vals.mean())
print(vals.std())

In [ ]:
plt.figure(figsize=(10, 10))
plt.hist(vals, bins=50)

# Text generation

In [ ]:
#signs=['aries','gemini','taurus','cancer','leo','virgo','libra','scorpio','sagittarius','capricorn','aquarius','pisces']
signs_r = ['овен', 'близнецы', 'телец', 'рак', 'лев', 'дева', 'весы', 'скорпион', 'стрелец', 'козерог', 'водолей',
           'рыбы']
sign_dict = {'aries': 'овен',
             'gemini': 'близнецы',
             'taurus': 'телец',
             'cancer': 'рак',
             'leo': 'лев',
             'virgo': 'дева',
             'libra': ' весы',
             'scorpio': 'скорпион',
             'sagittarius': 'стрелец',
             'capricorn': 'козерог',
             'aquarius': 'водолей',
             'pisces': 'рыбы'}

In [ ]:
model_path = 'models/astro_gpt3large'
tok = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.cuda()

In [ ]:
for sig in signs_r:
    text = "<s> " + sig
    inpt = tok.encode(text, return_tensors="pt")
    out = model.generate(inpt.cuda(),
                         max_length=100
                         ,
                         repetition_penalty=5.0,

                         do_sample=True, top_k=5,
                         top_p=0.95,
                         temperature=1)
    predict = tok.decode(out[0])
    print(sig.capitalize() + '\n' + str(predict).capitalize() + '\n')